# Citation Graph Builder Script (based on the python Scopus API)
Tom Aubier 2025

## 0. Get access to the Scopus API

Please consult the [Elsevier Developer Portal](https://dev.elsevier.com) to obtain a scopus API key.

You will have to specify this key when calling `pybliometrics.scopus.init()` in the following cell.

Please checkout the [pybliometrics documentation to learn more](https://pybliometrics.readthedocs.io/en/stable/).

In [ ]:
from pyvis.network import Network
from matplotlib.colors import LinearSegmentedColormap, to_hex
import pybliometrics, datetime
from tqdm import tqdm
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

pybliometrics.scopus.init()  # read API keys

## 1. Search for articles
Checkout [Scopus search documentation](https://schema.elsevier.com/dtds/document/bkapi/search/SCOPUSSearchTips.htm) to learn the search syntax and useful keywords.

Here is an example with some literature on calcium signaling in glia

In [3]:
query = 'TITLE-ABS-KEY(intercellular) AND TITLE-ABS-KEY(calcium wave) AND TITLE-ABS-KEY(glia)'

# doc: https://pybliometrics.readthedocs.io/en/stable/classes/ScopusSearch.html#pybliometrics.scopus.ScopusSearch
srch = pybliometrics.scopus.ScopusSearch(query, subscriber=False, download=False, verbose=True)
print(f'{srch.get_results_size()} articles found')

93 articles found


## 2. Download articles metadata
Search results are stored in a `pandas` `DataFrame` for easier handling.

In [4]:
srch = pybliometrics.scopus.ScopusSearch(query, subscriber=False, download=True, verbose=True)

scopus_df = pd.DataFrame(srch.results)
scopus_df.set_index('doi', inplace=True)
scopus_df['citation_dois'] = None

## 3. Get article citations tree
This step can take some time -> results are saved to a `pickle` file for later use

In [ ]:
for row_ii, contrib in tqdm(scopus_df.iterrows(), total=scopus_df.shape[0]):
    if contrib.citedby_count > 0 and 'eid' in contrib and isinstance(contrib.eid, str):
        try:
            citation_srch = pybliometrics.scopus.ScopusSearch(f'REF({contrib.eid})', subscriber=False, download=True, verbose=True)
            if citation_srch.results:
                scopus_df.at[row_ii, 'citation_dois'] = ' | '.join([contrib.doi for contrib in citation_srch.results if isinstance(contrib.doi, str)])
            # scopus_df.at[row_ii, 'citation_eids'] = citation_srch.get_eids()
        except pybliometrics.scopus.exception.ScopusQueryError:
            print(contrib)

scopus_df.to_pickle('example_data/scopus_citations_df.pkl')

100%|██████████| 93/93 [00:00<00:00, 314.94it/s]


## 4. Load previously pickled citations DataFrame
Irrelevant articles can optionally be filtered by title or DOI at this stage

In [ ]:
scopus_df = pd.read_pickle('example_data/scopus_citations_df.pkl')

# Ensures that None indices do not exist
scopus_df.index = [df_index if df_index is not None else ii for ii, (df_index, sc) in enumerate(scopus_df.iterrows())]

# Remove unappropriate contributions (by index)
dfidx2pop = [
    # 'doi to delete' # By DOI
    # scopus_df.index[scopus_df.title == 'Article title to be deleted'][0], # Title search if no DOI
]
for dfid2pop in dfidx2pop:
    if dfid2pop and dfid2pop in scopus_df.index:
        scopus_df.drop(index=dfid2pop, inplace=True)

scopus_df

eid                   pii  \
10.3390/ijms26125785                2-s2.0-105008938663                  None   
10.3390/ijms241311095                2-s2.0-85164845618                  None   
10.3389/fcell.2022.864599            2-s2.0-85128556685                  None   
10.1007/s11064-021-03255-8           2-s2.0-85100201006                  None   
10.1152/physrev.00043.2018           2-s2.0-85087132253                  None   
...                                                 ...                   ...   
10.1002/glia.440090202                2-s2.0-0027686469                  None   
10.1002/glia.440070203                2-s2.0-0027548734                  None   
10.1111/j.1471-4159.1992.tb09401.x    2-s2.0-0026736154                  None   
10.1016/0896-6273(91)90238-U          2-s2.0-0025823978  0896-6273(91)90238-U   
10.1016/0143-4160(91)90020-F          2-s2.0-0025921543      014341609190020F   

                                   pubmed_id  \
10.3390/ijms26125785                40565248   
10.3390/ijms241311095               37446272   
10.3389/fcell.2022.864599               None   
10.1007/s11064-021-03255-8          33523395   
10.1152/physrev.00043.2018          32326824   
...                                      ...   
10.1002/glia.440090202               8244537   
10.1002/glia.440070203               8094375   
10.1111/j.1471-4159.1992.tb09401.x   1629725   
10.1016/0896-6273(91)90238-U         1675864   
10.1016/0143-4160(91)90020-F         1647876   

                                                                                                                                                                 title  \
10.3390/ijms26125785                                                                                                   The Astroglia Syncytial Theory of Consciousness   
10.3390/ijms241311095                                  Ca2+ Dynamics of Gap Junction Coupled and Uncoupled Deiters’ Cells in the Organ of Corti in Hearing BALB/c Mice   
10.3389/fcell.2022.864599           Plasmalogens Regulate Retinal Connexin 43 Expression and Müller Glial Cells Gap Junction Intercellular Communication and Migration   
10.1007/s11064-021-03255-8                                                                                  Satellite Glial Cells and Astrocytes, a Comparative Review   
10.1152/physrev.00043.2018                                                                             Glial connexins and pannexins in the healthy and diseased brain   
...                                                                                                                                                                ...   
10.1002/glia.440090202                                                                                                                                   Glial calcium   
10.1002/glia.440070203                                           Mechanisms of intercellular calcium signaling in glial cells studied with dantrolene and thapsigargin   
10.1111/j.1471-4159.1992.tb09401.x                Activation of Protein Kinase C Blocks Astroglial Gap Junction Communication and Inhibits the Spread of Calcium Waves   
10.1016/0896-6273(91)90238-U                Intercellular signaling in glial cells: Calcium waves and oscillations in response to mechanical stimulation and glutamate   
10.1016/0143-4160(91)90020-F                                                                                                                  Ca2+ waves in astrocytes   

                                   subtype subtypeDescription  \
10.3390/ijms26125785                    re             Review   
10.3390/ijms241311095                   ar            Article   
10.3389/fcell.2022.864599               ar            Article   
10.1007/s11064-021-03255-8              ar            Article   
10.1152/physrev.00043.2018              ar            Article   
...                                    ...                ...   
10.100

## 5. Build interactive graph

In [6]:
# N citation colormap
colors = [
    (80/255, 107/255, 137/255),
    (165/255, 68/255, 70/255)]
br_labtau_cmap = LinearSegmentedColormap.from_list('labtau_cmap', colors, N=255)

node_relations = {doi: metadata.citation_dois if isinstance(metadata.citation_dois, list) else metadata.citation_dois.split(' | ') if isinstance(metadata.citation_dois, str) else None for doi, metadata in scopus_df.iterrows()}
node_datetimes = {doi: datetime.datetime.strptime(metadata.coverDate, '%Y-%m-%d') for doi, metadata in scopus_df.iterrows()}
max_ncitations = max([0 if citations is None else len(citations) for citations in node_relations.values()])

ingraph_node_relations = {
    node_doi: [cdois for cdois in cited_by_dois if cdois in node_relations.keys()] if isinstance(cited_by_dois, list) else None for node_doi, cited_by_dois in node_relations.items()
}
max_ingraph_ncitations = max([0 if citations is None else len(citations) for citations in ingraph_node_relations.values()])


net = Network(directed=True, filter_menu=False, select_menu=False, width='100%', height='800px')

# net.show_buttons() # Optional -> graph definition using GUI
net.show_buttons(filter_='physics')
net.set_options("""
const options = {
    "nodes": {
        "font": {"face": "'CMU Sans Serif', sans-serif"}
    },
    "edges": {
        "width": 1,
        "hoverWidth": 8,
        "selectionWidth": 12,
        "color": {
            "color": "lightgrey",
            "hover": "grey",
            "highlight": "black"
        }
    },
    "interaction": {
        "zoomSpeed": 0.2,
        "hover": true,
        "hoverConnectedEdges": true,
        "selectConnectedEdges": true,
        "multiselect": true,
        "keyboard": true
    }
}
""")

# Add nodes
# max_ncitations = 10 # clamp node size (optional)
for node_doi, ingraph_connections_dois in ingraph_node_relations.items():

    print('adding ', node_doi)

    metadata = scopus_df.loc[node_doi] # metadata.citedby_count
    ingraph_citation_count = len(ingraph_connections_dois) if ingraph_connections_dois else 0

    normalized_citation_count = metadata.citedby_count / max_ncitations
    normalized_ingraph_citation_count = ingraph_citation_count / max_ingraph_ncitations

    formated_doi = f'<br>DOI: <a href=\'https://doi.org/{node_doi}\'>{node_doi}' if isinstance(node_doi, str) else ''

    try:
        node = net.add_node(
            node_doi,
            fixed={'x': True, 'y': False},
            x=(node_datetimes[node_doi].timestamp() / 1.5e5),
            label=f'{metadata.creator} ({node_datetimes[node_doi].year})',
            title=f'{metadata.title}<br>First Author: {metadata.creator}<br>Publiation year: {node_datetimes[node_doi].year}<br>Citation count: {metadata.citedby_count:.0f} (in graph: {ingraph_citation_count:.0f}){formated_doi}',
            color=to_hex(br_labtau_cmap(normalized_citation_count)),
            size=15+80*normalized_citation_count,
            mass=.5+.5*normalized_citation_count,
            font={
                'size': 15 + 180*normalized_citation_count,
                'strokeColor': '#ffffff80',
                'strokeWidth': 30,
                'color': 'black',
            },
        )
    except TypeError as e:
        print(e)
        print(metadata)

# Add edges
for node_doi, connections_dois in node_relations.items():
    if connections_dois:
        for connected_node in connections_dois:
            if connected_node in node_relations:
                try:
                    net.add_edge(node_doi, connected_node, physics=False)
                except AssertionError as e:
                    print(f'Unable to add {node_doi} to {connections_dois} edge:\n\t-> {str(e)}\n\t-> Skipping edge')

net.save_graph('example_data/citation_graph.html') # Generate graph web page -> open with a web browser to explore graph


adding  10.3390/ijms26125785
adding  10.3390/ijms241311095
adding  10.3389/fcell.2022.864599
adding  10.1007/s11064-021-03255-8
adding  10.1152/physrev.00043.2018
adding  10.1111/jmi.12804
adding  10.1002/glia.23613
adding  10.1021/acschemneuro.8b00483
adding  10.1016/j.neulet.2017.06.035
adding  10.1523/JNEUROSCI.3906-15.2016
adding  10.1016/j.expneurol.2015.12.002
adding  10.1515/cmble-2015-0050
adding  10.3389/fncel.2015.00059
adding  10.1117/1.NPh.1.1.011012
adding  10.3389/fncom.2014.00045
adding  10.3233/JAD-130740
adding  10.1016/j.bbamcr.2014.04.016
adding  10.1007/s12035-013-8625-1
adding  10.1242/jcs.125617
adding  10.2217/fnl.13.38
adding  10.1016/j.pneurobio.2013.06.001
adding  10.3389/fneng.2013.00004
adding  10.1016/j.biocel.2013.05.024
adding  10.1016/j.neuint.2013.08.001
adding  10.1016/j.brainres.2012.06.046
adding  10.1016/j.brainres.2012.03.070
adding  10.1002/glia.22338
adding  10.1007/978-1-61779-452-0_33
adding  10.1111/j.1460-9568.2011.07901.x
adding  10.1109/TBM

Once opened using a web browser `example_data/citation_graph.html` should allow you to explore the citation graph in an interactive way.
![example_citation_graph_screenshot.png](example_data/example_citation_graph_screenshot.png)